In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('src/combined_data.csv')

df

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
...,...,...
83443,0,hi given a date how do i get the last date of ...
83444,1,now you can order software on cd or download i...
83445,1,dear valued member canadianpharmacy provides a...
83446,0,subscribe change profile contact us long term ...


In [6]:
import nltk, re, string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...


True

In [14]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [15]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [34]:
def preprocess_nltk(text, reg=r'[^a-zA-Z\s]'):
    text = re.sub(reg, '', text.lower())
    tokens = nltk.word_tokenize(text)
    return [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]

In [42]:
def preprocess_nltk_without_lem(text, reg=r'[^a-zA-Z\s]'):
    text = re.sub(reg, '', text.lower())
    tokens = nltk.word_tokenize(text)
    return [w for w in tokens if w not in stop_words]

Пример: "Buy now!!!" → ['buy', 'now'].

In [35]:
df['nltk_tokens'] = df['text'].apply(preprocess_nltk)

In [36]:
df['tokens_punk'] = df['text'].apply(lambda x: preprocess_nltk(x, reg=r'[^a-zA-Z0-9\s\.,;:!?()""\'\-\–—]'))

In [47]:
df['tokens_without_lem'] = df['text'].apply(lambda x: preprocess_nltk_without_lem(x, reg=r'[^a-zA-Z0-9\s\.,;:!?()""\'\-\–—]'))

In [41]:
df

,label,text,nltk_tokens,tokens_punk
0,1,ounce feather bowl hummingbird opec moment ala...,"[ounce, feather, bowl, hummingbird, opec, mome...","[ounce, feather, bowl, hummingbird, opec, mome..."
1,1,wulvob get your medircations online qnb ikud v...,"[wulvob, get, medircations, online, qnb, ikud,...","[wulvob, get, medircations, online, qnb, ikud,..."
2,0,computer connection from cnn com wednesday es...,"[computer, connection, cnn, com, wednesday, es...","[computer, connection, cnn, com, wednesday, es..."
3,1,university degree obtain a prosperous future m...,"[university, degree, obtain, prosperous, futur...","[university, degree, obtain, prosperous, futur..."
4,0,thanks for all your answers guys i know i shou...,"[thanks, answer, guy, know, checked, rsync, ma...","[thanks, answer, guy, know, checked, rsync, ma..."
...,...,...,...,...
83443,0,hi given a date how do i get the last date of ...,"[hi, given, date, get, last, date, month, data...","[hi, given, date, get, last, date, month, data..."
83444,1,now you can order software on cd or download i...,"[order, software, cd, download, site, immediat...","[order, software, cd, download, site, immediat..."
83445,1,dear valued member canadianpharmacy provides a...,"[dear, valued, member, canadianpharmacy, provi...","[dear, valued, member, canadianpharmacy, provi..."
83446,0,subscribe change profile contact us long term ...,"[subscribe, change, profile, contact, u, long,...","[subscribe, change, profile, contact, u, long,..."


In [19]:
from gensim.models import Word2Vec
import numpy as np

In [20]:
model = Word2Vec(df['nltk_tokens'].values, vector_size=100, window=5, min_count=1, workers=4)
def get_sentence_vector(text_tokens, model):
    vectors = [model.wv[w] for w in text_tokens if w in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

In [23]:
X = np.array([get_sentence_vector(t, model) for t in df['nltk_tokens']])
y = df['label']

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
clf = LogisticRegression()

In [27]:
clf.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [28]:
y_preds = clf.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [33]:
acc = accuracy_score(y_test, y_preds)
f1 = f1_score(y_test, y_preds)
prec = precision_score(y_test, y_preds)
recall = recall_score(y_test, y_preds)

print(f'acc = {acc}\nf1 = {f1}\nprecision_score = {prec}\nrecall_score = {recall}')

acc = 0.9687837028160575
f1 = 0.9704430702898962
precision_score = 0.9660040659588887
recall_score = 0.9749230593867548


In [43]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
forest = RandomForestClassifier(
    n_estimators=100,
    max_depth=10, 
    random_state=42)

forest.fit(X_train, y_train)

preds_forest = forest.predict(X_test)

In [45]:
acc = accuracy_score(y_test, preds_forest)
f1 = f1_score(y_test, preds_forest)
prec = precision_score(y_test, preds_forest)
recall = recall_score(y_test, preds_forest)

print(f'acc = {acc}\nf1 = {f1}\nprecision_score = {prec}\nrecall_score = {recall}')

acc = 0.5227082085080886
f1 = 0.6762315070720208
precision_score = 0.5236356769685907
recall_score = 0.954342090168636


In [37]:
X_p = np.array([get_sentence_vector(t, model) for t in df['tokens_punk']])

X_train_p, X_test_p, y_train, y_test = train_test_split(X_p, y, test_size=0.2)

In [39]:
clf_p = LogisticRegression()

clf_p.fit(X_train_p, y_train)

y_predic_p = clf_p.predict(X_test_p)

In [40]:
acc = accuracy_score(y_test, y_predic_p)
f1 = f1_score(y_test, y_predic_p)
prec = precision_score(y_test, y_predic_p)
recall = recall_score(y_test, y_predic_p)

print(f'acc = {acc}\nf1 = {f1}\nprecision_score = {prec}\nrecall_score = {recall}')

acc = 0.9672258837627322
f1 = 0.9686694541497222
precision_score = 0.9672843742850606
recall_score = 0.9700585063668693


In [52]:
X_wl = np.array([get_sentence_vector(t, model) for t in df['tokens_without_lem']])

X_train_wl, X_test_wl, y_train, y_test = train_test_split(X_p, y, test_size=0.2)

clf_wl = LogisticRegression()

clf_wl.fit(X_train_wl, y_train)

y_preds_wl = clf.predict(X_test_wl)

In [53]:
acc = accuracy_score(y_test, y_preds_wl)
f1 = f1_score(y_test, y_preds_wl)
prec = precision_score(y_test, y_preds_wl)
recall = recall_score(y_test, y_preds_wl)

print(f'acc = {acc}\nf1 = {f1}\nprecision_score = {prec}\nrecall_score = {recall}')

acc = 0.9688436189334931
f1 = 0.9707569452255089
precision_score = 0.9680349932705249
recall_score = 0.973494247687796


In [55]:
from nltk.stem import PorterStemmer

In [58]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

stemmer = PorterStemmer()

def preprocess_spacy(text):
    doc = nlp(re.sub(r'[^a-zA-Z\s]', '', text.lower()))
    return [stemmer.stem(token.text) for token in doc 
            if not token.is_stop and not token.is_punct and token.text]

In [59]:
df['spacy_tokens'] = df['text'].apply(preprocess_spacy)

In [60]:
df

,label,text,nltk_tokens,tokens_punk,tokens_without_lem,spacy_tokens
0,1,ounce feather bowl hummingbird opec moment ala...,"[ounce, feather, bowl, hummingbird, opec, mome...","[ounce, feather, bowl, hummingbird, opec, mome...","[ounce, feather, bowl, hummingbird, opec, mome...","[ounc, feather, bowl, hummingbird, opec, momen..."
1,1,wulvob get your medircations online qnb ikud v...,"[wulvob, get, medircations, online, qnb, ikud,...","[wulvob, get, medircations, online, qnb, ikud,...","[wulvob, get, medircations, online, qnb, ikud,...","[wulvob, medirc, onlin, qnb, ikud, viagra, esc..."
2,0,computer connection from cnn com wednesday es...,"[computer, connection, cnn, com, wednesday, es...","[computer, connection, cnn, com, wednesday, es...","[computer, connection, cnn, com, wednesday, es...","[ , comput, connect, cnn, com, wednesday, esca..."
3,1,university degree obtain a prosperous future m...,"[university, degree, obtain, prosperous, futur...","[university, degree, obtain, prosperous, futur...","[university, degree, obtain, prosperous, futur...","[univers, degre, obtain, prosper, futur, money..."
4,0,thanks for all your answers guys i know i shou...,"[thanks, answer, guy, know, checked, rsync, ma...","[thanks, answer, guy, know, checked, rsync, ma...","[thanks, answers, guys, know, checked, rsync, ...","[thank, answer, guy, know, check, rsync, manua..."
...,...,...,...,...,...,...
83443,0,hi given a date how do i get the last date of ...,"[hi, given, date, get, last, date, month, data...","[hi, given, date, get, last, date, month, data...","[hi, given, date, get, last, date, month, data...","[hi, given, date, date, month, data, form, yyy..."
83444,1,now you can order software on cd or download i...,"[order, software, cd, download, site, immediat...","[order, software, cd, download, site, immediat...","[order, software, cd, download, site, immediat...","[order, softwar, cd, download, site, immedi, \..."
83445,1,dear valued member canadianpharmacy provides a...,"[dear, valued, member, canadianpharmacy, provi...","[dear, valued, member, canadianpharmacy, provi...","[dear, valued, member, canadianpharmacy, provi...","[dear, valu, member, canadianpharmaci, provid,..."
83446,0,subscribe change profile contact us long term ...,"[subscribe, change, profile, contact, u, long,...","[subscribe, change, profile, contact, u, long,...","[subscribe, change, profile, contact, us, long...","[subscrib, chang, profil, contact, long, term,..."


In [ ]:
nlp_md = spacy.load("en_core_web_md")

def get_token_vectors(text):
    doc = nlp(text)
    vectors = [token.vector for token in doc if token.has_vector]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)

df['mean_token_vector'] = df['text'].apply(get_token_vectors)
X_spacy = np.stack(df['mean_token_vector'].values) 

In [ ]:
X_spacy[0]